In [20]:
import pandas as pd
df = pd.read_csv('exam112.csv')
df.describe()
#print(df.columns)

,no,nclass,hours,extra,math,eng,science,score
count,112.000000,112.000000,112.000000,112.000000,111.000000,111.000000,111.000000,112.000000
mean,56.500000,2.883929,19.455357,6.107143,68.657658,63.531532,69.477477,201.482143
std,32.475632,1.431597,10.138579,2.326336,13.713627,13.713842,13.594014,37.829866
min,1.000000,1.000000,3.000000,0.000000,37.000000,35.000000,29.000000,101.000000
25%,28.750000,2.000000,12.000000,5.000000,62.000000,55.000000,61.500000,179.000000
50%,56.500000,3.000000,17.000000,6.000000,67.000000,61.000000,67.000000,194.500000
75%,84.250000,4.000000,25.000000,7.000000,74.000000,68.000000,79.000000,215.000000
max,112.000000,5.000000,43.000000,12.000000,102.000000,97.000000,102.000000,298.000000


In [21]:
df['pass'] = df['pass'].map({'Pass': 1, 'Fail': 0})

df.fillna(df.mean(), inplace=True)  # 평균값으로 채우기
df.fillna(df.median(), inplace=True)  # 중앙값으로 채우기
df.fillna(df.mode().iloc[0], inplace=True)  # 최빈값으로 채우기

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# 이상치 판별 기준
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치 확인
outliers = (df < lower_bound) | (df > upper_bound)
print(outliers.sum())  # 각 열에서 이상치 개수 확인

df = df[~outliers.any(axis=1)]  # 이상치가 포함된 행 제거

df

no          0
nclass      0
hours       0
extra      10
math       13
eng        11
science     1
score      12
pass        0
dtype: int64


,no,nclass,hours,extra,math,eng,science,score,pass
0,1,1,28,6,68.657658,73.000000,65.0,205,1
1,2,1,18,7,70.000000,63.531532,73.0,217,1
3,4,1,19,6,63.000000,65.000000,66.0,194,1
4,5,1,15,4,63.000000,49.000000,60.0,172,0
5,6,1,17,6,74.000000,58.000000,74.0,206,1
...,...,...,...,...,...,...,...,...,...
103,104,2,8,5,60.000000,61.000000,60.0,181,0
104,105,3,13,3,47.000000,51.000000,51.0,149,0
105,106,4,9,4,54.000000,59.000000,58.0,171,0
106,107,4,12,5,59.000000,57.000000,63.0,179,0


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


# 데이터 불러오기
df = pd.read_csv("exam112.csv")

# '합격여부'를 숫자로 변환 (Pass → 1, Fail → 0)
df['pass'] = df['pass'].map({'Pass': 1, 'Fail': 0})

# 결측치 처리 (평균값으로 채우기)
df.fillna(df.mean(), inplace=True)

# 이상치 처리 (IQR 방식으로 제거)
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

# 특성(X)과 타겟(y) 설정
X_reg = df[['math']]  # 회귀 모델 (수학 → 과학 점수 예측)
y_reg = df['science']

X_clf = df[['math', 'eng', 'science']]  # 분류 모델 (수학+영어+과학 → 합격 여부 예측)
y_clf = df['pass']

# 데이터 분할
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# 데이터 정규화 (KNN은 거리 기반이므로 스케일링 필요)
scaler_reg = StandardScaler()
X_reg_train = scaler_reg.fit_transform(X_reg_train)
X_reg_test = scaler_reg.transform(X_reg_test)

scaler_clf = StandardScaler()
X_clf_train = scaler_clf.fit_transform(X_clf_train)
X_clf_test = scaler_clf.transform(X_clf_test)

# KNN 회귀 모델 (수학 → 과학 점수 예측)
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_reg_train, y_reg_train)

# KNN 분류 모델 (수학+영어+과학 → 합격 여부 예측)
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_clf_train, y_clf_train)

def predict_science():
    math_score = float(input("예측할 수학 점수를 입력하세요: "))
    math_score = np.array(math_score).reshape(-1, 1)  # 2D 배열로 변환
    math_score = scaler_reg.transform(math_score)  # 스케일링 적용
    predicted_science = knn_reg.predict(math_score)  # 예측
    print(f"예측된 과학 점수: {predicted_science[0]:.2f}")


def predict_pass():
    math_score = float(input("수학 점수를 입력하세요: "))
    english_score = float(input("영어 점수를 입력하세요: "))
    science_score = float(input("과학 점수를 입력하세요: "))

    input_scores = np.array([[math_score, english_score, science_score]])  # 2D 배열 유지
    input_scores = scaler_clf.transform(input_scores)  # 스케일링 적용
    predicted_pass = knn_clf.predict(input_scores)  # 예측
    result = "Pass" if predicted_pass[0] == 1 else "Fail"

    print(f"예측된 결과: {result}")


# 실행
print("1: 수학 점수 → 과학 점수 예측")
print("2: 수학, 영어, 과학 점수 → 합격 여부 예측")
choice = input("실행할 기능을 선택하세요 (1/2): ")

if choice == "1":
    predict_science()
elif choice == "2":
    predict_pass()
else:
    print("잘못된 입력입니다.")


1: 수학 점수 → 과학 점수 예측
2: 수학, 영어, 과학 점수 → 합격 여부 예측
실행할 기능을 선택하세요 (1/2): 1
예측할 수학 점수를 입력하세요: 38
예측된 과학 점수: 48.80
